In [24]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from flask import Flask, render_template, request
from PIL import Image


In [25]:
dataset_path = 'datasets'
saved_model_path = 'model_wrinkle.h5'


In [26]:
image_size = (64, 64)
batch_size = 32
epochs = 10


In [27]:
# Create the mouth detection model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [28]:
def preprocess_dataset(dataset_path):
    # Preprocess the dataset by resizing images and creating label directories
    open_dir = os.path.join(dataset_path, 'NoWrinkles')
    closed_dir = os.path.join(dataset_path, 'Wrinkled')

    if not os.path.exists(open_dir):
        os.makedirs(open_dir)
    if not os.path.exists(closed_dir):
        os.makedirs(closed_dir)

    for filename in os.listdir(dataset_path):
        if filename.endswith('.jpg'):
            if 'class0' in filename:
                image_dir = open_dir
            else:
                image_dir = closed_dir

            img = cv2.imread(os.path.join(dataset_path, filename))
            img = cv2.resize(img, image_size)
            cv2.imwrite(os.path.join(image_dir, filename), img)

preprocess_dataset(dataset_path)

In [29]:
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)




Found 853 images belonging to 2 classes.


Found 212 images belonging to 2 classes.


In [30]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[ModelCheckpoint(saved_model_path, save_best_only=True)]
)

Epoch 1/20
26/26 [==============================] - 3s 100ms/step - loss: 0.6234 - accuracy: 0.6480 - val_loss: 0.6408 - val_accuracy: 0.4948
Epoch 2/20
26/26 [==============================] - 2s 92ms/step - loss: 0.4260 - accuracy: 0.7909 - val_loss: 0.5269 - val_accuracy: 0.6719
Epoch 3/20
26/26 [==============================] - 2s 93ms/step - loss: 0.3927 - accuracy: 0.8185 - val_loss: 0.5048 - val_accuracy: 0.6667
Epoch 4/20
26/26 [==============================] - 2s 93ms/step - loss: 0.3481 - accuracy: 0.8441 - val_loss: 0.4477 - val_accuracy: 0.6771
Epoch 5/20
26/26 [==============================] - 3s 112ms/step - loss: 0.3507 - accuracy: 0.8404 - val_loss: 0.3755 - val_accuracy: 0.8490
Epoch 6/20
26/26 [==============================] - 3s 113ms/step - loss: 0.3155 - accuracy: 0.8551 - val_loss: 0.3674 - val_accuracy: 0.8594
Epoch 7/20
26/26 [==============================] - 3s 110ms/step - loss: 0.3242 - accuracy: 0.8660 - val_loss: 0.5470 - val_accuracy: 0.6406
Epoch 8/2

In [31]:
model.evaluate(validation_generator)

7/7 [==============================] - 0s 28ms/step - loss: 0.3370 - accuracy: 0.8821


[0.33700037002563477, 0.8820754885673523]

In [ ]:
model_json = model.to_json()
with open("model_wrinkle.json","w") as json_file:
    json_file.write(model_json)